# Use Random forest
* this notebook will use random forest to run classification on the data.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from mne.decoding import Vectorizer
from sklearn.preprocessing import MinMaxScaler
from mne.decoding import SPoC
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Ridge
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split as tts
import mne
from sklearn.model_selection import StratifiedKFold
from mne.datasets import sample
from mne.decoding import (SlidingEstimator, GeneralizingEstimator,
                          cross_val_multiscore, LinearModel, get_coef)
from sklearn.decomposition import PCA
from sklearn.decomposition import NMF
from sklearn.manifold import TSNE 
import os
from mne.decoding import UnsupervisedSpatialFilter
import os.path as op
from os import listdir
from os.path import isfile, join
from collections import defaultdict
from scipy.stats import skew, kurtosis
import mne 

from mne.decoding import Vectorizer
from sklearn.preprocessing import MinMaxScaler, Normalizer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.svm import SVC  # noqa
from sklearn.model_selection import ShuffleSplit  # noqa

from mne.decoding import UnsupervisedSpatialFilter
from sklearn.decomposition import PCA
from scipy.stats import skew, kurtosis
from sklearn.metrics import confusion_matrix
from mne.viz import tight_layout

import time
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
%matplotlib inline

import time
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold, cross_val_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from urllib.request import urlopen

plt.style.use('ggplot')
pd.set_option('display.max_columns', 500)

In [2]:
%run Batch_ArtifactFilter_Epoch.ipynb

In [3]:
epoch=mne.read_epochs('D:\\USB Drive\\NewEEG-200s\\New\\20131216_1310_epo.fif')

Reading D:\USB Drive\NewEEG-200s\New\20131216_1310_epo.fif ...
    Found the data of interest:
        t =       0.00 ...     500.00 ms
        0 CTF compensation matrices available
1759 matching events found
Applying baseline correction (mode: mean)
1759 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 projection items activated


In [4]:
def filter_event_id(epoch):
    newepoch=epoch
    epochs=[[]]
    j=0
    for i in range(len(newepoch.events[:,-1])):
        if i % 3 ==0:
            epochs.append([])
            j=j+1
            epochs[j].append(epoch.events[i,-1])
        else:
            epochs[j].append(epoch.events[i,-1])
    for events in epochs:
        if  4 in events or 1 in events or 13 in events or 16 in events:
            for i in range(len(events)):
                    events[i]=4
        else:
            for i in range(len(events)):
                    events[i]=5
    epochs.pop(0)
    flattened_list = [y for x in epochs for y in x]
    for i in range(len(newepoch.events)):
        newepoch.events[i]=flattened_list[i]
    return newepoch

In [5]:
epoch=filter_event_id(epoch)
print(epoch.events[:,-1])

[4 4 4 ... 4 4 4]


In [6]:
epoch.pick_channels(['LB3','R2','L12','LC4','LC1','LD4','RR11','L14','L1','RE4','R1','RR1','RC7','R3','LC3','STI 014'])

<EpochsFIF  |   1759 events (all good), 0 - 0.5 sec, baseline [0, 0], ~55.3 MB, data loaded,
 'b-f1-pic': 0
 'b-f1-snd': 0
 'b-f1-spk': 948
 'b-f1-wrd': 0
 'b-f2-pic': 0
 'b-f2-snd': 0
 'b-f2-spk': 811
 'b-f2-wrd': 0
 'b-tg-pic': 0
 'b-tg-snd': 0
 'b-tg-spk': 0
 'b-tg-wrd': 0
 'f-f1-pic': 0
 'f-f1-snd': 0
 'f-f1-spk': 0
 'f-f1-wrd': 0
 'f-f2-pic': 0
 'f-f2-snd': 0
 'f-f2-spk': 0
 'f-f2-wrd': 0
 'f-tg-pic': 0
 'f-tg-snd': 0
 'f-tg-spk': 0
 'f-tg-wrd': 0>

In [9]:
from sklearn import metrics
clf = RandomForestRegressor(n_estimators =10, random_state = 42)
    
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.20, random_state = 42)
    
clf.fit(X_train, y_train)
    
y_pred=clf.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred.round()))

Accuracy: 0.8835227272727273


In [8]:
epoch.rename_channels({'STI 014' : 'test channel'})

In [7]:
X = epoch.get_data()
y = epoch.events[:,-1]
vec=Vectorizer()
X=vec.fit_transform(X)
print(X.shape)

(1759, 4112)


In [10]:
channel=[]
importances = np.array(clf.feature_importances_)
for i in range(0,len(importances),257):
    channel.append(np.sum(importances[i:i+257]))
    channel1=np.array(channel)
    channel1.shape
channel1.shape
import pandas as pd
feature_imp = pd.Series(channel1,index=epoch.ch_names).sort_values(ascending=False)
feature_imp

test channel    0.698451
L14             0.045902
L12             0.044073
L1              0.026055
LC1             0.025756
R2              0.022756
R1              0.020869
RE4             0.019525
R3              0.018621
RC7             0.015730
LB3             0.014206
RR11            0.013593
LC3             0.010118
LC4             0.009359
LD4             0.007572
RR1             0.007414
dtype: float64